In [1]:
import tensorflow as tf
import utils.dataset as dset
import utils.model as bModel

# Baseline CNN model
Let's define a baseline CNN model using well known best practices.


## Load and compile the model


In [2]:
baseline = bModel.get_model()
baseline.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 48, 48, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 44, 44, 32)        2432      
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 44, 44, 32)        0         
_________________________________________________________________
instance_normalization (Inst (None, 44, 44, 32)        64        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 42, 42, 64)        18496     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 42, 42, 64)        0         
_________________________________________________________________
instance_normalization_1 (In (None, 42, 42, 64)        128   

The model is compiled using binary crossentropy as loss function, Adam optimizer and accuracy as metric.

In [3]:
baseline.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-4),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics='binary_accuracy')

In [4]:
# Callbacks for early stopping and reducing learning rate on plateau
# Both monitor validation loss
callbacks = [
    tf.keras.callbacks.ReduceLROnPlateau(factor=0.1, patience=5, monitor='val_loss'),
    tf.keras.callbacks.EarlyStopping(min_delta=1e-2, patience=10, monitor='val_loss')
]

## Load the dataset for training

In [5]:
ds_train, ds_valid = dset.load_train_val_dataset()

# Preprocessing
ds_train = dset.preprocess_train(ds_train, batch_size=64)
ds_valid = dset.preprocess_validation(ds_valid)

## Training

In [6]:
baseline.fit(ds_train, epochs=50,
             validation_data=ds_valid, callbacks=callbacks)


Epoch 1/50
4096/4096 [==============================] - 61s 14ms/step - loss: 0.4426 - binary_accuracy: 0.7779 - val_loss: 0.3988 - val_binary_accuracy: 0.8173
Epoch 2/50
4096/4096 [==============================] - 59s 14ms/step - loss: 0.3567 - binary_accuracy: 0.8369 - val_loss: 0.3871 - val_binary_accuracy: 0.8147
Epoch 3/50
4096/4096 [==============================] - 61s 15ms/step - loss: 0.3274 - binary_accuracy: 0.8545 - val_loss: 0.3775 - val_binary_accuracy: 0.8201
Epoch 4/50
4096/4096 [==============================] - 61s 15ms/step - loss: 0.3080 - binary_accuracy: 0.8651 - val_loss: 0.3841 - val_binary_accuracy: 0.8335
Epoch 5/50
4096/4096 [==============================] - 61s 15ms/step - loss: 0.2941 - binary_accuracy: 0.8720 - val_loss: 0.3702 - val_binary_accuracy: 0.8228
Epoch 6/50
4096/4096 [==============================] - 61s 15ms/step - loss: 0.2841 - binary_accuracy: 0.8772 - val_loss: 0.3496 - val_binary_accuracy: 0.8394
Epoch 7/50
4096/4096 [==================

## Save the model

In [7]:
from utils.dataset import base_path as cwd

baseline.save_weights(f"{cwd}/Baseline CNN/bin/baselineCNN.h5")